# Cek Apakah passage masih memiliki \n\n
\n\n adalah separator antara passage, yg mana jika tidak ada berarti model llm hanya menerima input teks berupa passage pertama saja, dan bukan passage kedua & ketika

In [ ]:
from datasets import load_from_disk
from transformers import AutoTokenizer
from tqdm import tqdm

# Load dataset dan tokenizer
dataset_name = "khalidrizki//RECOMP-tuning"
model_name = "./models/-google-flan-t5-base-2025-06-09_19-36-13"
dataset = load_from_disk(dataset_name)
test_data = dataset["test"]
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Inisialisasi
decoded_passages = []
pipe_counts = []
multi_pipe_count = 0
single_or_none_pipe_count = 0

# Proses setiap passage
for passage in tqdm(test_data["passages"]):
    # Tokenisasi dan truncation
    tokenized = tokenizer(
        passage,
        max_length=512,
        truncation=True,
        return_tensors="pt",
        add_special_tokens=True
    )
    
    # Decode
    decoded = tokenizer.decode(tokenized["input_ids"][0], skip_special_tokens=True)
    decoded_passages.append(decoded)

    # Hitung jumlah karakter pipe
    pipe_count = decoded.count("|")
    pipe_counts.append(pipe_count)

    # Klasifikasi
    if pipe_count > 1:
        multi_pipe_count += 1
    else:
        single_or_none_pipe_count += 1

# Tambahkan kolom ke dataset
test_data = test_data.add_column("truncated_decoded_passage", decoded_passages)
test_data = test_data.add_column("pipe_count", pipe_counts)

# Hitung total dan persentase
total = multi_pipe_count + single_or_none_pipe_count
percent_multi = 100 * multi_pipe_count / total
percent_single_or_none = 100 * single_or_none_pipe_count / total

# Cetak statistik
print(f"Total entries: {total}")
print(f"Pipe > 1: {multi_pipe_count} ({percent_multi:.2f}%)")
print(f"Pipe ≤ 1: {single_or_none_pipe_count} ({percent_single_or_none:.2f}%)")

# # Contoh hasil
# print("\nContoh hasil:")
# for i in range(3):
#     print(f"\nOriginal passage:\n{test_data['passages'][i]}")
#     print(f"\nTruncated-decoded passage:\n{test_data['truncated_decoded_passage'][i]}")
#     print(f"Jumlah karakter '|': {test_data['pipe_count'][i]}")


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
seeded_and_with_selection = "./models/-google-flan-t5-base-2025-06-09_19-36-13"
t5_seeded_with_selection = AutoModelForSeq2SeqLM.from_pretrained(seeded_and_with_selection)
t5_sws_tokenizer = AutoTokenizer.from_pretrained(seeded_and_with_selection)
t5_seeded_with_selection.push_to_hub('khalidrizki/T5base-RECOMP-seededDataset-withSelection')
t5_sws_tokenizer.push_to_hub('khalidrizki/T5base-RECOMP-seededDataset-withSelection')

model.safetensors: 100%|██████████| 990M/990M [12:59<00:00, 1.27MB/s]  


CommitInfo(commit_url='https://huggingface.co/khalidrizki/T5base-RECOMP-seededDataset-withSelection/commit/0ae16a2cd84260598315c767c2130f589dd45333', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='0ae16a2cd84260598315c767c2130f589dd45333', pr_url=None, repo_url=RepoUrl('https://huggingface.co/khalidrizki/T5base-RECOMP-seededDataset-withSelection', endpoint='https://huggingface.co', repo_type='model', repo_id='khalidrizki/T5base-RECOMP-seededDataset-withSelection'), pr_revision=None, pr_num=None)

# Generate Answer based on T5 Summary (dgn target=final_summary)

In [1]:
from datasets import load_dataset
dataset = load_dataset("khalidrizki/RECOMP-tuning")
test_data = dataset["test"]

from utils import load_model_and_tokenizer
model_name='Qwen/Qwen3-1.7B'
model, tokenizer, config = load_model_and_tokenizer(model_name)

c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model on cuda with torch.bfloat16


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 12.73it/s]


Di bawah, aku akan melakukan pengujian terhadap beberapa kasus berbeda:
1. Model yg dilatih thd dataset yg telah di-seed dan dataset latihan tsb ada seleksinya `(khalidrizki/T5base-RECOMP-seededDataset-withSelection)`
2. Model yg dilatih thd dataset yg telah di-seed dan dataset latihan tsb tiada seleksi
3. Model yg dilatih thd dataset yg belum di-seed dan dataset latihan tsb ada seleksi
4. Model yg dilatih thd dataset yg belum di-seed dan dataset latihan tsb tiada seleksi `(khalidrizki/T5base-RECOMP-unseedDataset-noSelection)`

Perangkuman oleh kompresor sudah dijalankan dengan command `python train_summarizer.py --model_name_or_path khalidrizki/{TERGANTUNG PAKAI MODEL APA} --do_predict --dataset_name khalidrizki/RECOMP-tuning --max_target_length 52 --output_dir ./outputs/ --per_device_eval_batch_size=32 --predict_with_generate --summary_column final_summary --seed 42`

Seharusnya: antar model yg sama2 melakukan with/no seleksi (tetapi hanya dibedakan oleh seed dataset saja) akan memiliki skor yang mirip. 

__Ukuran dataset kurang?__: antar model yg sama-sama un/seeded (tetapi berbeda pada diterapkan atau tidaknya seleksi), model yg dgn seleksi seharusnya lebih baik. Kalau tidak lebih baik, berarti ukuran dataset kurang. 

Idealnya

## T5base-RECOMP-seededDataset-withSelection

Sebelum menjalankan generasi jawaban dengan rangkuman hasil fine tuning T5, T5 sudah melakukan generasi rangkuman ketika menjalankan train_summarizer.py dengan command `python train_summarizer.py --do_predict`

In [3]:
# Muat baris-baris teks dari file
with open("./outputs/-khalidrizki-T5base-RECOMP-seededDataset-withSelection_on-recompTuning_hub_dataset/generated_predictions.txt", "r", encoding="utf-8", errors='replace') as f:
    predictions = [line.strip() for line in f.readlines()]

# Pastikan jumlah prediksi sama dengan jumlah data
assert len(predictions) == len(test_data), f"Jumlah prediksi ({len(predictions)}) tidak sama dengan jumlah data ({len(test_data)})"

AssertionError: Jumlah prediksi (564) tidak sama dengan jumlah data (565)

In [4]:
while len(predictions) < len(test_data):
    print('jumlah prediksi kurang, meng-append string kosong ("")')
    predictions.append("")

# Pastikan jumlah prediksi sama dengan jumlah data
assert len(predictions) == len(test_data), f"Jumlah prediksi ({len(predictions)}) tidak sama dengan jumlah data ({len(test_data)})"

# Tambahkan kolom ke dataset
test_data = test_data.add_column("T5_summary", predictions)

from generate_answer import generate_answer_and_do_scoring

answer_based_on_t5_summary = generate_answer_and_do_scoring(
    test_data, 
    query_col='query', 
    summary_col="T5_summary", 
    label_col='answer', 
    passages_col='passages', 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=52
)

jumlah prediksi kurang, meng-append string kosong ("")


Generating responses with summary only:   0%|          | 0/565 [00:00<?, ?it/s]c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset

In [5]:
from datasets import Dataset
result = Dataset.from_list(answer_based_on_t5_summary)
em_avg = sum(result['em'])/len(result)
f1_avg =  sum(result['f1'])/len(result)
print("EM:" ,em_avg)
print("F1:", f1_avg)

EM: 0.10973451327433628
F1: 0.08678890872044633


In [ ]:

em_avg

0.11504424778761062

In [ ]:
f1_avg

0.0948697836281932

## T5base-RECOMP-unseedDataset-noSelection

In [6]:
# Muat baris-baris teks dari file
with open("./outputs/-khalidrizki-T5base-RECOMP-unseedDataset-noSelection_on-recompTuning_hub_dataset/generated_predictions.txt", "r", encoding="utf-8", errors='replace') as f:
    predictions_unselective = [line.strip() for line in f.readlines()]

# Pastikan jumlah prediksi sama dengan jumlah data
assert len(predictions_unselective) == len(test_data), f"Jumlah prediksi ({len(predictions_unselective)}) tidak sama dengan jumlah data ({len(test_data)})"

# Tambahkan kolom ke dataset
test_data = test_data.add_column("unselective_summary", predictions_unselective)

from generate_answer import generate_answer_and_do_scoring

answer_based_on_unselective_summary = generate_answer_and_do_scoring(
    test_data, 
    query_col='query', 
    summary_col="unselective_summary", 
    label_col='answer', 
    passages_col='passages', 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=52
)

from datasets import Dataset
result_uns = Dataset.from_list(answer_based_on_unselective_summary)
em_uns_avg = sum(result_uns['em'])/len(result_uns)
f1_uns_avg =  sum(result_uns['f1'])/len(result_uns)
print("EM:" ,em_uns_avg)
print("F1:", f1_uns_avg)

Generating responses with summary only:   0%|          | 0/565 [00:00<?, ?it/s]c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset

EM: 0.4247787610619469
F1: 0.2505083083585362


## RAG konvensional

In [7]:
from generate_answer import generate_answer_and_do_scoring
answer_based_on_passages = generate_answer_and_do_scoring(
    test_data, 
    query_col='query', 
    summary_col="passages", 
    label_col='answer', 
    passages_col='passages', 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=52, 
    max_source_length = 512
)

from datasets import Dataset
normal_RAG = Dataset.from_list(answer_based_on_passages)
print("HASIL RAG BIASA")
print("rerata EM:", sum(normal_RAG['em'])/len(normal_RAG))
print("rerata F1:", sum(normal_RAG['f1'])/len(normal_RAG))

Generating responses with summary only: 100%|██████████| 565/565 [35:01<00:00,  3.72s/it]

HASIL RAG BIASA
rerata EM: 0.32035398230088497
rerata F1: 0.1765652985106173


# Comparison (tidak diperlukan)

In [3]:
result_1 = "./outputs/-.-models--google-flan-t5-base-2025-05-28_07-17-31-2025-05-28_10-26-57/generated_predictions.txt"
result_2 = "./outputs/-khalidrizki-T5base-RECOMP-unseedDataset-noSelection_on-recompTuning_hub_dataset/generated_predictions.txt"
output_diff = "perbedaan.txt"

# Baca semua baris dari kedua file
with open(result_1, "r", encoding="utf-8") as f1, open(result_2, "r", encoding="utf-8") as f2:
    lines1 = [line.rstrip('\n') for line in f1]
    lines2 = [line.rstrip('\n') for line in f2]

# Pastikan jumlah baris sama
min_len = min(len(lines1), len(lines2))
same_count = 0
diff_count = 0

with open(output_diff, "w", encoding="utf-8") as out:
    for i in range(min_len):
        if lines1[i] == lines2[i]:
            same_count += 1
        else:
            diff_count += 1
            out.write(f"[Baris {i+1} berbeda]\n")
            out.write(f"result_1: {lines1[i]}\n")
            out.write(f"result_2: {lines2[i]}\n\n")

    if len(lines1) != len(lines2):
        out.write("[Jumlah baris berbeda antara kedua file]\n")
        out.write(f"Total baris result_1: {len(lines1)}\n")
        out.write(f"Total baris result_2: {len(lines2)}\n")

# Cetak ringkasan
print(f"Jumlah baris sama    : {same_count}")
print(f"Jumlah baris berbeda : {diff_count}")


Jumlah baris sama    : 465
Jumlah baris berbeda : 100


In [ ]:
from datasets import load_dataset
ds = load_dataset('khalidrizki/RECOMP-tuning')
test = ds['test']

In [ ]:
test[0]

# Percobaan pada dataset lama (tidak di-seed)

In [ ]:
from datasets import Dataset
result_of_unseed_dataset_with_no_selective_summarization = ("./outputs/EM_F1_testing/SCORES_google-flan-t5-base-2025-05-28_07-17-31")
fin = load_dataset(result_of_unseed_dataset_with_no_selective_summarization)

In [24]:
sum(fin['em'])/len(fin)

0.42123893805309737

In [50]:
sum(substringmatch_xcode_details)

226.0

In [51]:
len(substringmatch_xcode_details)

565

In [25]:
sum(fin['f1'])/len(fin)

0.25522817089095584

## Menguji kinerja jika menggunakan RAG biasa

In [ ]:
from generate_answer import generate_answer_and_do_scoring
answer_based_on_passages = generate_answer_and_do_scoring(
    test_data, 
    query_col='query', 
    summary_col="passages", 
    label_col='answer', 
    passages_col='passages', 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=52, 
    max_source_length = 512
)

In [8]:
from datasets import Dataset
normal_RAG = Dataset.from_list(answer_based_on_passages)
print("HASIL RAG BIASA")
print("rerata EM:", sum(normal_RAG['em'])/len(normal_RAG))
print("rerata F1:", sum(normal_RAG['f1'])/len(normal_RAG))

HASIL RAG BIASA
rerata EM: 0.29557522123893804
rerata F1: 0.1659491847867348


__Terbukti bahwa dengan menggunakan RECOMP kinerja RAG menjadi lebih baik, bisa dilihat dari skor EM dan F1 nya yang lebih tinggi__

# Push Model ke Huggingface Hub

In [18]:
t5_model_path = "./models/-google-flan-t5-base-2025-05-28_07-17-31"

In [20]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_path)
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_path)

In [ ]:
hub_repo = "khalidrizki/indonesian-T5-RECOMP"
t5_model.push_to_hub(hub_repo)
t5_tokenizer.push_to_hub(hub_repo)